# Basic Example: Multiclass Closed QN with Class Switching

This example demonstrates a complex closed queueing network:
- **Nodes**: Delay node + PS Queue
- **Classes**: 3 job classes with different service distributions
- **Routing**: Complex class switching patterns between stations

The model showcases various distribution types and class switching.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun example_jline_closedModel_3(): Network {
    """
    Create a multiclass closed QN with class switching.
    
    Features:
    - Delay node with Erlang, HyperExp, and Exp service
    - PS Queue with HyperExp and Exp service
    - Complex class switching routing
    """
    val model = Network("model")
// Block 1: nodes
    val delay = Delay(model, "Delay")
    val queue = Queue(model, "Queue1", SchedStrategy.PS)
// Block 2: classes
    val jobclass1 = ClosedClass(model, "Class1", 2, delay, 0)
    val jobclass2 = ClosedClass(model, "Class2", 0, delay, 0)
    val jobclass3 = ClosedClass(model, "Class3", 1, delay, 0)
// Service times at Delay node
    delay.setService(jobclass1, Erlang(3, 2))
    delay.setService(jobclass2, HyperExp(0.5, 3.0, 10.0))
    delay.setService(jobclass3, Exp(1))
// Service times at Queue
    queue.setService(jobclass1, HyperExp(0.1, 1.0, 10.0))
    queue.setService(jobclass2, Exp(2))
    queue.setService(jobclass3, Exp(3))
// Block 3: topology with class switching
// This creates a complex routing pattern between classes
    val P = model.initRoutingMatrix()
// Class 1 routing
    P.addRoute(jobclass1, delay, delay, 0.3)
    P.addRoute(jobclass1, delay, queue, 0.6)
    P.addClassSwitch(jobclass1, jobclass2, delay, delay, 0.1)
    P.addRoute(jobclass1, queue, delay, 0.2)
    P.addClassSwitch(jobclass1, jobclass2, queue, queue, 0.8)
// Class 2 routing  
    P.addClassSwitch(jobclass2, jobclass1, delay, queue, 1.0)
    P.addClassSwitch(jobclass2, jobclass1, queue, delay, 1.0)
// Class 3 routing (simple circular)
    P.addRoute(jobclass3, delay, queue, 1.0)
    P.addRoute(jobclass3, queue, delay, 1.0)
    
    model.link(P)
    
    return model
// Create the model
val model = example_jline_closedModel_3()

## About This Model

**Service distributions**:
- **Erlang(3, 2)**: 2-phase Erlang with rate 3
- **HyperExp(0.5, 3.0, 10.0)**: Mixture with p=0.5, rates 3.0 and 10.0
- **Exp(μ)**: Exponential with rate μ

**Class switching**:
- Class 1 jobs can become Class 2 jobs
- Class 2 jobs become Class 1 jobs
- Class 3 jobs circulate independently

This creates complex interactions between classes.

In [ ]:
// Solve with MVA
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
println(f"\nSOLVER: {solver_mva.name}")
val avg_table = solver_mva.avgTable
println(avg_table)